In [2]:
pip install transformers

     |████████████████████████████████| 757kB 3.5MB/s 
     |████████████████████████████████| 3.0MB 7.0MB/s 
     |████████████████████████████████| 890kB 29.7MB/s 
     |████████████████████████████████| 1.1MB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c484d24562e8b276624ee40834ce5943ce32337f92db5718595b19bbe82ec4bf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib

max_len = 384
#configuration = BertConfig()

In [4]:
DATA_IN_PATH = 'data_in/KOR'
DATA_OUT_PATH = "data_out/KOR"

In [5]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['var_'+_string, ''])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [6]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
save_path = "bert-base-multilingual-uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-uncased/vocab.txt", lowercase=True)

In [7]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

3883008/3881058 [==============================] - 0s 0us/step


In [8]:
!wget -P ./bert-base-multilingual-uncased/ https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json

--2020-07-06 04:47:19--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.105.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.105.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [application/json]
Saving to: ‘./bert-base-multilingual-uncased/bert-base-multilingual-uncased-config.json’

bert-base-multiling 100%[===================>]     625  --.-KB/s    in 0s      

2020-07-06 04:47:19 (9.99 MB/s) - ‘./bert-base-multilingual-uncased/bert-base-multilingual-uncased-config.json’ saved [625/625]



In [9]:
!mv ./bert-base-multilingual-uncased/bert-base-multilingual-uncased-config.json ./bert-base-multilingual-uncased/config.json 

In [10]:
!wget -P ./bert-base-multilingual-uncased/ https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-tf_model.h5

--2020-07-06 04:48:11--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-tf_model.h5
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.129.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.129.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 999358484 (953M) [binary/octet-stream]
Saving to: ‘./bert-base-multilingual-uncased/bert-base-multilingual-uncased-tf_model.h5’

bert-base-multiling 100%[===================>] 953.06M  39.2MB/s    in 21s     

2020-07-06 04:48:33 (44.6 MB/s) - ‘./bert-base-multilingual-uncased/bert-base-multilingual-uncased-tf_model.h5’ saved [999358484/999358484]



In [11]:
!mv ./bert-base-multilingual-uncased/bert-base-multilingual-uncased-tf_model.h5 ./bert-base-multilingual-uncased/tf_model.h5

In [12]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)
        #print(tokenized_context)
        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

60407 training points created.
5774 evaluation points created.


In [13]:
class TFBERTKorquad(tf.keras.Model):
    def __init__(self, max_len=None):
        super(TFBERTKorquad, self).__init__()
        self.encoder = TFBertModel.from_pretrained('./bert-base-multilingual-uncased/')
        #self.encoder = TFBertModel.from_pretrained("bert-base-multilingual-uncased")
        self.start_logit = tf.keras.layers.Dense(1, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(1, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(inputs, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)

        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = tf.keras.layers.Activation(tf.keras.activations.softmax)(start_logits)
        end_probs = tf.keras.layers.Activation(tf.keras.activations.softmax)(end_logits)
    
        return start_probs, end_probs

In [14]:
korquad_model = TFBERTKorquad(max_len)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Some weights of the model checkpoint at ./bert-base-multilingual-uncased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-uncased/.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [16]:
class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_answer(pred_ans)
            normalized_true_ans = [normalize_answer(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [17]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [18]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [19]:
model_name = "tf2_bert_korquad"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

data_out/KOR/tf2_bert_korquad -- Folder create complete 



In [20]:
history = korquad_model.fit(
    x_train,
    y_train,
    epochs=3,  # For demonstration, 3 epochs are recommended
    verbose=2,
    batch_size=4, #2->64
    callbacks=[exact_match_callback, earlystop_callback, cp_callback]
)

Epoch 1/3

epoch=1, exact match score=0.06
7788/7788 - 6010s - loss: 7.6960 - output_1_loss: 3.7383 - output_2_loss: 3.9577
Epoch 2/3

epoch=2, exact match score=0.06
7788/7788 - 6049s - loss: 6.7806 - output_1_loss: 3.4303 - output_2_loss: 3.3503
Epoch 3/3

epoch=3, exact match score=0.07
7788/7788 - 6043s - loss: 6.1601 - output_1_loss: 3.2470 - output_2_loss: 2.9131


In [21]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

KeyError: ignored